In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os

In [11]:
DIMENSION_THLD = 8
ASSETS_PATH = 'doc/assets'
SCALE = 4

In [12]:
data = pd.read_pickle(os.path.join(".", "out-dct", "data-result.pkl"))
data = data.query('Dimension >= 3')
data["LogTime"] = data["Time"].apply(np.log2)
data.head()

,Time,Run,Type,Dimension,LogTime
0,0.004807,1,HomeMade DCT,4,-7.700720
1,0.040110,1,HomeMade DCT,5,-4.639890
2,0.270744,1,HomeMade DCT,6,-1.884998
3,2.001292,1,HomeMade DCT,7,1.000931
4,16.371788,1,HomeMade DCT,8,4.033140


In [13]:
valid_data = data.query(f'Dimension >= {DIMENSION_THLD} or Type == "HomeMade DCT"')
invalid_data = data.query(f'Dimension < {DIMENSION_THLD} and Type == "Numpy FFT"')
assert len(valid_data) + len(invalid_data) == len(data)
invalid_data.head()

,Time,Run,Type,Dimension,LogTime
1,0.000069,1,Numpy FFT,3,-13.825074
2,0.000080,1,Numpy FFT,4,-13.611983
3,0.000110,1,Numpy FFT,5,-13.151377
4,0.000132,1,Numpy FFT,6,-12.883656
5,0.000336,1,Numpy FFT,7,-11.539544


In [27]:
fig = px.scatter(valid_data, x="Dimension", y="LogTime", color="Type", trendline="ols")

fig.add_trace(go.Scatter(x=invalid_data["Dimension"], y=invalid_data["LogTime"],
                         mode='markers', name="Numpy FFT (low dim)"))

full_fig = fig.full_figure_for_development(warn=False)
min_y, max_y = full_fig.layout.yaxis.range
min_y, max_y = int(min_y), int(max_y)
min_x, max_x = full_fig.layout.xaxis.range
min_x, max_x = int(min_x), int(max_x)

fig.update_layout(
    title= dict(
        text=f'Numpy FFT vs Custom DCT time comparison',
        xanchor='center',
        x=0.5),
    xaxis_title = 'Matrix dimension',
    yaxis_title = 'Time (s)',
    yaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(min_y, max_y + 1, 2)],
        ticktext = ['$10^{}$'.format('{' + str(x) + '}') for x in range(min_y, max_y + 1, 2)],
        #ticktext = [10**x for x in range(min_y, max_y + 1)],
    ),
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x for x in range(min_x, max_x + 1)],
        ticktext = ['$2^{}$'.format('{' + str(x) + '}') for x in range(min_x, max_x + 1)],
        #ticktext = [10**x for x in range(min_y, max_y + 1)],
    ),
    legend=dict(title="Method"),
)
fig.update_traces(marker=dict(color='#EF553B'), showlegend=False, selector=dict(name="Numpy FFT (low dim)"))
fig.update_traces(name="Custom DCT", selector=dict(name="HomeMade DCT"))

fig.show()


fig.write_image(os.path.join(ASSETS_PATH, 'dct2.png'), scale=SCALE)

In [15]:
def extract_regression_parameters(target_fig):
    trend_df = px.get_trendline_results(target_fig)
    trend_df["m"] = trend_df["px_fit_results"].apply(lambda x : x.params[1])
    trend_df["q"] = trend_df["px_fit_results"].apply(lambda x : x.params[0])
    return trend_df

regression_parameters = extract_regression_parameters(fig)
regression_parameters

,Type,px_fit_results,m,q
0,HomeMade DCT,<statsmodels.regression.linear_model.Regressio...,2.959939,-19.587254
1,Numpy FFT,<statsmodels.regression.linear_model.Regressio...,2.209918,-26.693555


In [28]:
time_ratio = min(regression_parameters.m) / max(regression_parameters.m)
print(f"Theoretical time ratio (2 / 3): {2 / 3}")
print(f"Time ratio: {time_ratio}")

Theoretical time ratio (2 / 3): 0.6666666666666666
Time ratio: 0.7466091993677608
